## 自定义一个可迭代对象

一个可迭代对像需要有一个 `[Symbol.iterator]` 的方法，这个方法需要返回一个迭代器。

迭代器就是包含一个 `next` 方法的对象，`next` 方法返回的值为一个对象，需要有 `{done: boolean, value: T}`。done 为 true 就就意味着迭代结束，为 false 就会取 value 的值。

可以查看 MDN 的 [iteration protocol](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Iteration_protocols#the_iterable_protocol) 定义

In [ ]:
let countTen = {
  [Symbol.iterator] () {
    return {
      from: 1,
      to: 10,
  
      next() {
        if( this.from <= this.to ) {
          return {done: false, value: this.from++}
        }
        return {done: true, value: undefined}
      }
    }
  }
};

可以通过可迭代对象的迭代器生成函数得到该对象的迭代器，调用迭代器的 next 方法可以得到其返回的值。这里得到的 next 返回的完整的信息，即 `{done: true, value: 1}`。

In [ ]:
const iterator = countTen[Symbol.iterator]();
iterator.next()

而当我们用 `for...of` 遍历可迭代对象时，item 拿到的只是 value 的值，并且在 done 为 true 的时候直接跳出了循环。

In [ ]:
for (const item of countTen) {
  console.log(item);
}

## `for...of` 为我们做了什么

揭开了迭代器的面纱，我们可以从大致推测 `for...of` 在逻辑上帮我们做了什么，下面自定义一个 `forOf`。可以看出就是帮我们不断地调用迭代器的 next 方法，并且取到值，当 done 为 true 的时候退出循环。

In [ ]:
function forOf<T> (iterable: Iterable<T>, callback:(item: T)=> void) {
  const iterate = iterable[Symbol.iterator]();
  while(true) {
    const res = iterate.next();
    if(res.done) {
      break;
    }
    const item = res.value;
    callback(item)
  }
}

自定义的 forOf 函数与 `for...of` 有类似的行为

In [ ]:
forOf(countTen, item => { console.log(item); });

## generate 函数

通过 [MDN 对 Generator 的定义](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Generator)。可以粗略的理解 generator 函数的返回值是一个可迭代的迭代器。

In [ ]:
function *countTenGen(){
  for(let i = 1; i <= 10; i++) {
    yield i;
  }
}

直接调用 generator 函数返回值的 next 可以得到一个 `{done: boolean, value: T}` 类型的值，说明它的返回值是一个 Iterator。

In [ ]:
countTenGen().next()

这里通过 `for...of` 直接操作一个 generator 函数的返回值，发现也可以成功的遍历。说明 generator 函数的返回值是一个可遍历的对象，即满足 Iterable protocol。

In [ ]:
for (const item of countTenGen()) {
  console.log(item);
}

这里再用我们自己定义的 forOf 函数来试试，神奇的是一样成功的打印出了 1 到 10。再次说明其返回值是一个满足 Iterable protocol 的值。

In [ ]:
forOf(countTenGen(), item => console.log(item));

那么这是怎么做到的呢？
从下面的代码可以看出，就是给迭代器对象设置了 `[Symbol.iterator]` 方法，通过这个方法返回了迭代器自身

In [ ]:
let iterableIterator = countTenGen();
iterableIterator === iterableIterator[Symbol.iterator](); // true

我们可以从内置的数组类型上看到一样的情况

In [ ]:
const arr = [1, 2, 3];
const arrIterator = arr[Symbol.iterator]();
arrIterator === arrIterator[Symbol.iterator](); // true


In [ ]:
// 不管调用多少次 [Symbol.iterator] 方法都可以遍历
for (const item of arr[Symbol.iterator]()[Symbol.iterator]()) {
  console.log(item);
}

再回过头看开始自定义的可迭代对象，用 `for..of` 尝试操作它的迭代器，会报错。

```js
for (const item of countTen[Symbol.iterator]()) {
  // 会报错
}
```

因为我们自动以的 countTen 的迭代器生成函数返回的只是一个 Iterator，并不是一个 Iterable 的对象

对之前的 countTen 做一点小小的修改，再运行上面会报错的代码，会发现可以成功打印出 1 到 10 了。

In [ ]:
countTen = {
  [Symbol.iterator] () {
    return {
      from: 1,
      to: 10,
  
      next() {
        if( this.from <= this.to ) {
          return {done: false, value: this.from++}
        }
        return {done: true, value: undefined}
      },

      [Symbol.iterator]() {
        return this;
      }
    }
  }
}

In [ ]:
for (const item of countTen[Symbol.iterator]()) {
  console.log(item)
}